## Exploring the Sem Eval 2018 dataset using Autolabel

#### Set up the API keys for the providers you want to use

In [ ]:
import os

# provide your own OpenAI API key here
os.environ['OPENAI_API_KEY'] = 'sk-xxxxxxxxxxxxxxxxx'

#### Install the autolabel library

In [ ]:
!pip install 'refuel-autolabel[openai]'

#### Download the dataset

In [1]:
from autolabel import get_data

get_data('twitter_emotion_detection')

2023-06-27 15:46:06 autolabel.utils ERROR: twitter_emotion_detection not in list of available datasets: ['banking', 'civil_comments', 'ledgar', 'walmart_amazon', 'company', 'squad_v2', 'sciq', 'conll2003', 'movie_reviews']. Exiting...


This downloads two datasets:
* `test.csv`: This is the larger dataset we are trying to label using LLMs
* `seed.csv`: This is a small dataset where we already have human-provided labels

## Start the labeling process!

Labeling with Autolabel is a 3-step process:
* First, we specify a labeling configuration (see `config.json` below)
* Next, we do a dry-run on our dataset using the LLM specified in `config.json` by running `agent.plan`
* Finally, we run the labeling with `agent.run`

In [2]:
import json

from autolabel import LabelingAgent

In [3]:
# load the config
with open('config_twitter_emotion_detection.json', 'r') as f:
    config = json.load(f)

Let's review the configuration file below. You'll notice the following useful keys:
* `task_type`: `multi_label_classification` (since it's a multi label classification task)
* `model`: `{'provider': 'openai', 'name': 'gpt-3.5-turbo'}` (use a specific OpenAI model)
* `prompt.labels`: `['neutral', 'anger', 'anticipation', 'disgust', 'fear', ...]` (the full list of labels to choose from)
* `prompt.task_guidelines`: `'You are an expert at classifying tweets as...` (how we describe the task to the LLM)
* `prompt.few_shot_num`: 5 (how many labeled examples to provide to the LLM)

In [4]:
config

{'task_name': 'EmotionClassification',
 'task_type': 'multi_label_classification',
 'dataset': {'label_column': 'label', 'delimiter': ','},
 'model': {'provider': 'openai', 'name': 'gpt-3.5-turbo'},
 'prompt': {'task_guidelines': 'You are an expert at classifying tweets as neutral or one or more of the given emotions that best represent the mental state of the poster.\nYour job is to correctly label the provided input example into one or more of the following categories:\n{labels}',
  'output_guidelines': 'You will return the answer as a comma separated list of labels sorted in alphabetical order. For example: "label1, label2, label3"',
  'labels': ['neutral',
   'anger',
   'anticipation',
   'disgust',
   'fear',
   'joy',
   'love',
   'optimism',
   'pessimism',
   'sadness',
   'surprise',
   'trust'],
  'few_shot_examples': 'seed.csv',
  'few_shot_selection': 'semantic_similarity',
  'few_shot_num': 5,
  'example_template': 'Input: {example}\nOutput: {label}'}}

In [5]:
# create an aggent for labeling
agent = LabelingAgent(config=config)

In [6]:
agent.plan('test.csv')

Output()

┌──────────────────────────┬─────────┐
│ Total Estimated Cost     │ $8.0995 │
│ Number of Examples       │ 3259    │
│ Average cost per example │ $0.0025 │
└──────────────────────────┴─────────┘

───────────────────────────────────────────────── Prompt Example ──────────────────────────────────────────────────

You are an expert at classifying tweets as neutral or one or more of the given emotions that best represent the mental state of the poster.
Your job is to correctly label the provided input example into one or more of the following categories:
neutral
anger
anticipation
disgust
fear
joy
love
optimism
pessimism
sadness
surprise
trust

You will return the answer as a comma separated list of labels sorted in alphabetical order. For example: "label1, label2, label3"

Some examples with their output answers are provided below:

Input: @MaryamNSharif I think just becz u have so much terror in pak nd urself being  a leader u forgot d difference btw a leader nd terrorist !
Output: anger, disgust, fear

Input: In wake of fresh #terror threat and sounding of alert in #Mumbai, praying for safety &amp; security of everybody in the city #Maharashtra #news
Output: fear

Input: Somewhere I rd a rpt tht Pakis wr afraid of TSD &amp; asked it 2 shut dn. Congis obliged &amp; exposed it,hounded them.time 

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [7]:
# now, do the actual labeling
labels, df, metrics_list = agent.run('test.csv', max_items=100)

2023-06-27 15:46:37 autolabel.labeler INFO: Task run already exists.


There is an existing task with following details: id='819886949' created_at=datetime.datetime(2023, 6, 27, 15, 40, 
36, 211499) task_id='df73f1a5af5377e2c5d32a538912194b' dataset_id='22d508fa35f12dab54f000de60c2e219' 
current_index=35 output_file='test_labeled.csv' status=<TaskStatus.ACTIVE: 'active'> error=None metrics=None

Evaluating the existing task...

┏━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ f1     ┃ support ┃ accuracy ┃ completion_rate ┃
┡━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ 0.4475 │ 40      │ 0.0      │ 1.0             │
└────────┴─────────┴──────────┴─────────────────┘

35 examples labeled so far.

...

Deleted the existing task and starting a new one...

Output()

Actual Cost: 0.0302


┏━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ f1     ┃ support ┃ accuracy ┃ completion_rate ┃
┡━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ 0.4889 │ 100     │ 0.08     │ 1.0             │
└────────┴─────────┴──────────┴─────────────────┘

Total number of failures: 0

We are at a 0.4889 f1 when labeling the first 100 examples. Let's see if we can use confidence scores to improve f1 further by removing the less confident examples from our labeled set.

### Compute confidence scores

In [8]:
# Start computing confidence scores (using Refuel's LLMs)
os.environ['REFUEL_API_KEY'] = 'sk-xxxxxxxxxxxxxxxxx'

In [9]:
# set `compute_confidence` -> True
config["model"]["compute_confidence"] = True

In [10]:
agent = LabelingAgent(config=config)

In [11]:
agent.plan('test.csv')

Output()

┌──────────────────────────┬─────────┐
│ Total Estimated Cost     │ $8.0997 │
│ Number of Examples       │ 3259    │
│ Average cost per example │ $0.0025 │
└──────────────────────────┴─────────┘

───────────────────────────────────────────────── Prompt Example ──────────────────────────────────────────────────

You are an expert at classifying tweets as neutral or one or more of the given emotions that best represent the mental state of the poster.
Your job is to correctly label the provided input example into one or more of the following categories:
neutral
anger
anticipation
disgust
fear
joy
love
optimism
pessimism
sadness
surprise
trust

You will return the answer as a comma separated list of labels sorted in alphabetical order. For example: "label1, label2, label3"

Some examples with their output answers are provided below:

Input: @MaryamNSharif I think just becz u have so much terror in pak nd urself being  a leader u forgot d difference btw a leader nd terrorist !
Output: anger, disgust, fear

Input: In wake of fresh #terror threat and sounding of alert in #Mumbai, praying for safety &amp; security of everybody in the city #Maharashtra #news
Output: fear

Input: Somewhere I rd a rpt tht Pakis wr afraid of TSD &amp; asked it 2 shut dn. Congis obliged &amp; exposed it,hounded them.time 

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [12]:
labels, df, metrics_list = agent.run('test.csv', max_items=100)

Output()

2023-06-27 15:50:48 autolabel.confidence WARNING: Retrying autolabel.confidence.ConfidenceCalculator._call_with_retry in 2.0 seconds as it raised HTTPError: 429 Client Error: Too Many Requests for url: https://refuel-llm.refuel.ai/.


2023-06-27 15:51:11 autolabel.confidence WARNING: Retrying autolabel.confidence.ConfidenceCalculator._call_with_retry in 2.0 seconds as it raised HTTPError: 429 Client Error: Too Many Requests for url: https://refuel-llm.refuel.ai/.


2023-06-27 15:51:16 autolabel.confidence WARNING: Retrying autolabel.confidence.ConfidenceCalculator._call_with_retry in 2.0 seconds as it raised HTTPError: 429 Client Error: Too Many Requests for url: https://refuel-llm.refuel.ai/.


2023-06-27 15:51:20 autolabel.confidence WARNING: Retrying autolabel.confidence.ConfidenceCalculator._call_with_retry in 2.0 seconds as it raised HTTPError: 429 Client Error: Too Many Requests for url: https://refuel-llm.refuel.ai/.


...

2023-06-27 15:52:27 autolabel.confidence WARNING: Retrying autolabel.confidence.ConfidenceCalculator._call_with_retry in 2.0 seconds as it raised HTTPError: 429 Client Error: Too Many Requests for url: https://refuel-llm.refuel.ai/.


2023-06-27 15:52:30 autolabel.confidence WARNING: Retrying autolabel.confidence.ConfidenceCalculator._call_with_retry in 2.0 seconds as it raised HTTPError: 429 Client Error: Too Many Requests for url: https://refuel-llm.refuel.ai/.


Metric: auroc: 0.7582
Actual Cost: 0.0029


┏━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┓
┃ f1     ┃ support ┃ accuracy ┃ completion_rate ┃ threshold ┃
┡━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━┩
│ 0.6667 │ 1       │ 0.0      │ 0.01            │ 0.9555    │
│ 0.8333 │ 2       │ 0.5      │ 0.02            │ 0.9248    │
│ 0.7143 │ 7       │ 0.1429   │ 0.07            │ 0.8177    │
│ 0.75   │ 8       │ 0.25     │ 0.08            │ 0.8099    │
│ 0.7467 │ 10      │ 0.2      │ 0.1             │ 0.7784    │
│ 0.7697 │ 11      │ 0.2727   │ 0.11            │ 0.7746    │
│ 0.7611 │ 12      │ 0.25     │ 0.12            │ 0.765     │
│ 0.7795 │ 13      │ 0.3077   │ 0.13            │ 0.7606    │
│ 0.6276 │ 29      │ 0.1379   │ 0.29            │ 0.6927    │
│ 0.64   │ 30      │ 0.1667   │ 0.3             │ 0.6805    │
│ 0.6442 │ 36      │ 0.1389   │ 0.36            │ 0.6147    │
│ 0.6538 │ 37      │ 0.1622   │ 0.37            │ 0.6114    │
│ 0.6084 │ 50      │ 0.12     │ 0.5             │ 0.5561    │
│ 0.6161 │ 51      │ 0.1373   │ 0.51            │ 0.5553    │
│ 0.5651 │ 61      │ 0.1148   │ 0.61            │ 0.5011    │
│ 0.5721 │ 62      │ 0.129    │ 0.62            │ 0.4988    │
│ 0.4889 │ 100     │ 0.08     │ 1.0             │ 0.0206    │
└────────┴─────────┴──────────┴─────────────────┴───────────┘

Total number of failures: 0

Looking at the table above, we can see that if we set the confidence threshold at `0.6805`, we are able to label at 0.65 f1 and getting a completion rate of 30%. This means, we would ignore all the data points where confidence score is less than `0.6805` (which would end up being around 70% of all samples). This would, however, guarantee a higher quality labeled dataset for us.